# Assignment 4: Unsupervised Learning Clustering and Recommendations
## Group 105
- Natasa Bolic (300241734)
- Brent Palmer (300193610)
## Imports

## Introduction

## Dataset Description

## Loading Data and Basic Exploration

## Data Preparation

## Studies

## Conclusion

## References